In [1]:
!pip install torch torchvision torchaudio


In [3]:
import numpy as np
import torch
print(torch.__version__)


2.6.0+cpu


 # Linear regression using pytorch

In [12]:
# again the input is temp,rainfall, humidity
inputs = np.array([
    [ 50,  62, 105],
    [ 87,  51, 137],
    [118,  78, 120],
    [ 88,  71, 121],
    [ 89, 109,  44]], dtype= 'float32')

In [10]:
#targets are apples oranges 
targets = np.array([
    [150, 200],  # apples yield: 150, oranges yield: 200 for the first row
    [180, 220],  # apples yield: 180, oranges yield: 220 for the second row
    [160, 210],  # apples yield: 160, oranges yield: 210 for the third row
    [170, 190],  # apples yield: 170, oranges yield: 190 for the fourth row
    [165, 195]   # apples yield: 165, oranges yield: 195 for the fifth row
], dtype= 'float32')


In [13]:
# convert inputs and targets to tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [19]:
# now what we are doing is to geneare the random bais and weights

w = torch.randn(2,3, requires_grad = True)
b = torch.randn(2, requires_grad = True)

print(w)
print(b)

tensor([[-0.1348, -0.9038, -0.3303],
        [ 0.8751, -0.3124, -1.3438]], requires_grad=True)
tensor([-0.3643,  1.2211], requires_grad=True)


In [21]:
# x multiply with w transpose and add with b  also @ represent matrix multiplication pytorch

def model(x):
    return x@ w.t() +b

In [22]:
#generate predication

preds = model(inputs)
print(preds)

tensor([[ -97.8155, -115.4991],
        [-103.4297, -122.6876],
        [-126.3957,  -81.1512],
        [-116.3558, -106.5599],
        [-125.4039,  -14.0822]], grad_fn=<AddBackward0>)


In [23]:
# just to see the targets to compare it with the predication how close it is 

print(targets)

tensor([[150., 200.],
        [180., 220.],
        [160., 210.],
        [170., 190.],
        [165., 195.]])


In [32]:
# before imporve the model we need to evalaute our model and that is the loss  and the below is mean sqaured error

diff =preds - targets
torch.sum(diff*diff)/diff.numel()   #element wise multiplication

tensor(82350.7969, grad_fn=<DivBackward0>)

In [33]:
def mse (t1,t2):
    diff =t1 - t2
    return torch.sum(diff* diff)/ diff.numel()

In [34]:
loss = mse(preds, targets)
print(loss)

tensor(82350.7969, grad_fn=<DivBackward0>)


In [35]:
# compute the gradients
loss.backward()


In [36]:
# gradients for weights
print (w)
print(w.grad)

tensor([[-0.1348, -0.9038, -0.3303],
        [ 0.8751, -0.3124, -1.3438]], requires_grad=True)
tensor([[-24377.8223, -20828.7246, -29328.9609],
        [-24930.0430, -20718.7031, -32019.4238]])


In [38]:
# adjust the weights bias using gradient descent 
# 1. generate predicaion
# 2. calculate the loss
# 3. compute gradients w.r.t the weights and biases
# 4. adjust the wieghts by subtracting a small quantity proportinal to the gradient
# 5. reset the gradient to zero

preds = model(inputs)
print(preds)


tensor([[ -97.8155, -115.4991],
        [-103.4297, -122.6876],
        [-126.3957,  -81.1512],
        [-116.3558, -106.5599],
        [-125.4039,  -14.0822]], grad_fn=<AddBackward0>)


In [39]:
#calculate the loss

loss = mse(preds, targets)
print(loss)

tensor(82350.7969, grad_fn=<DivBackward0>)


In [40]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-48755.6445, -41657.4492, -58657.9219],
        [-49860.0859, -41437.4062, -64038.8477]])
tensor([-557.7602, -581.9920])


In [43]:
# adjust weights ad reset the gradients

with torch.no_grad():
    w -= w.grad *1e-5
    b -= b.grad *1e-5
    w.grad.zero_()
    b.grad.zero_()

In [44]:
print(w)
print(b)

tensor([[1.3279, 0.3459, 1.4295],
        [2.3709, 0.9307, 0.5773]], requires_grad=True)
tensor([-0.3476,  1.2386], requires_grad=True)


In [49]:
#claculate loss
preds =model(inputs)
loss = mse(preds, targets)

print(loss)

tensor(19616.2637, grad_fn=<DivBackward0>)


In [67]:
 # train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad *1e-5
        b -= b.grad *1e-5
        w.grad.zero_()
        b.grad.zero_()
        


In [68]:
#calculaet the loss

preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(170.2846, grad_fn=<DivBackward0>)


In [69]:
#predications
preds

tensor([[157.9981, 197.0208],
        [161.2015, 200.8887],
        [173.8664, 212.5071],
        [173.4805, 214.2707],
        [157.0566, 188.0333]], grad_fn=<AddBackward0>)

In [70]:
targets

tensor([[150., 200.],
        [180., 220.],
        [160., 210.],
        [170., 190.],
        [165., 195.]])

 # linear regression using pytroch built-ins

In [71]:
import torch.nn as nn

In [76]:
inputs = np.array([
    [50,  62, 105],[91,88,64],[87,135,58],
    [87,  51, 137],[69,97,78],[73,67,43],
    [118,  78, 120],[87,134,58],[102,43,38],
    [88,  71, 121],[73,67,43],[91,88,64],
    [89, 109,  44],[102,43,55],[69,96,70]],dtype= 'float32')

In [77]:
targets = np.array([
    [120, 75], [140, 80], [110, 65],
    [130, 85], [115, 70], [125, 95],
    [145, 100], [135, 90], [120, 85],
    [110, 65], [125, 70], [115, 60],
    [130, 75], [140, 85], [150, 100]
], dtype='float32')


In [78]:
# convert inputs and targets to tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [81]:
# Importing the TensorDataset class from PyTorch's torch.utils.data module.
# TensorDataset is used to wrap input and target tensors into a dataset object, making it easier to work with them in training.
# It allows you to bundle the features (inputs) and labels (targets) together, so they can be accessed in a single object for training.
# This is useful when you want to efficiently manage and iterate through your data for model training.

from torch.utils.data import TensorDataset

In [83]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 50.,  62., 105.],
         [ 91.,  88.,  64.],
         [ 87., 135.,  58.]]),
 tensor([[120.,  75.],
         [140.,  80.],
         [110.,  65.]]))

In [86]:
# Importing the DataLoader class from PyTorch's torch.utils.data module.
# DataLoader is used to load and iterate over datasets in batches, which is useful for training models.
# It helps to efficiently manage large datasets by splitting them into smaller batches and optionally shuffling them.
# DataLoader also handles the parallel loading of data (via the num_workers parameter) and supports features like batching and shuffling.
# This is essential when training models as it improves memory usage and training speed by loading smaller portions of data at a time.

from torch.utils.data import DataLoader


In [87]:

batch_size = 5
train_dl = DataLoader(train_ds,batch_size, shuffle = True)

In [88]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 88.,  71., 121.],
        [ 73.,  67.,  43.],
        [ 87., 135.,  58.],
        [ 89., 109.,  44.]])
tensor([[135.,  90.],
        [110.,  65.],
        [125.,  95.],
        [110.,  65.],
        [130.,  75.]])


In [92]:
#define model
# Define a simple neural network model with one linear layer.
# nn.Linear(input_features, output_features)
model =nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1594, -0.1415, -0.2368],
        [-0.5615, -0.4372, -0.2729]], requires_grad=True)
Parameter containing:
tensor([ 0.5543, -0.4733], requires_grad=True)


In [93]:
list(model.parameters())  #contins lisst and contins the weights and bias

[Parameter containing:
 tensor([[-0.1594, -0.1415, -0.2368],
         [-0.5615, -0.4372, -0.2729]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5543, -0.4733], requires_grad=True)]

In [94]:
#genearte predication
preds = model(inputs)
print(preds)

tensor([[ -41.0522,  -84.3111],
        [ -41.5600, -107.5102],
        [ -46.1535, -124.1756],
        [ -52.9703, -109.0104],
        [ -42.6415, -102.9130],
        [ -30.7463,  -82.4907],
        [ -57.7081, -133.5818],
        [ -46.0119, -123.7383],
        [ -30.7886,  -86.9165],
        [ -52.1719, -113.9494],
        [ -30.7463,  -82.4907],
        [ -41.5600, -107.5102],
        [ -39.4778, -110.1103],
        [ -34.8138,  -91.5561],
        [ -40.6058, -100.2925]], grad_fn=<AddmmBackward0>)


In [95]:
#loss functiom
import torch.nn.functional as F

#define the loss function
loss_fn = F.mse_loss

In [96]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(31573.5254, grad_fn=<MseLossBackward0>)


In [97]:
#optimizer

#define the optimizwr 
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [115]:
# Function to train the model for a specified number of epochs
def fit(num_epochs, model, loss_fn, opt, train_dl):
    # Iterate over the number of epochs (e.g., 100)
    for epoch in range(num_epochs):
        # Iterate over batches of data from the training data loader (train_dl)
        for xb, yb in train_dl:
            # Forward pass: compute the predicted values using the model
            pred = model(xb)
            
            # Compute the loss (error) between predicted values and actual target values (yb)
            loss = loss_fn(pred, yb)
            
            # Backpropagate the error (compute gradients for the model parameters)
            loss.backward()
            
            # Update the model parameters using the optimizer
            opt.step()
            
            # Zero the gradients after the update to prepare for the next iteration
            opt.zero_grad()
        
        # Every 10 epochs, print the current epoch and the loss value
        print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
            

In [117]:
fit(400, model, loss_fn, opt, train_dl)

Epoch [1/400], Loss:473.7966
Epoch [2/400], Loss:279.5580
Epoch [3/400], Loss:144.7926
Epoch [4/400], Loss:421.1958
Epoch [5/400], Loss:423.2240
Epoch [6/400], Loss:499.7539
Epoch [7/400], Loss:277.2403
Epoch [8/400], Loss:466.9181
Epoch [9/400], Loss:168.4995
Epoch [10/400], Loss:162.2524
Epoch [11/400], Loss:254.2240
Epoch [12/400], Loss:550.2134
Epoch [13/400], Loss:163.2447
Epoch [14/400], Loss:328.2859
Epoch [15/400], Loss:537.1932
Epoch [16/400], Loss:225.8770
Epoch [17/400], Loss:306.0306
Epoch [18/400], Loss:404.5529
Epoch [19/400], Loss:269.7166
Epoch [20/400], Loss:174.3310
Epoch [21/400], Loss:361.7229
Epoch [22/400], Loss:109.4663
Epoch [23/400], Loss:509.7591
Epoch [24/400], Loss:474.1230
Epoch [25/400], Loss:281.4485
Epoch [26/400], Loss:344.8466
Epoch [27/400], Loss:290.5508
Epoch [28/400], Loss:359.0347
Epoch [29/400], Loss:465.5396
Epoch [30/400], Loss:530.6081
Epoch [31/400], Loss:188.7867
Epoch [32/400], Loss:415.6129
Epoch [33/400], Loss:387.2535
Epoch [34/400], Los

In [118]:
preds = model(inputs)
preds

tensor([[ 91.0131,  56.7913],
        [130.2580,  81.5608],
        [139.4335,  84.9063],
        [130.6523,  84.1496],
        [114.5048,  70.4503],
        [101.7566,  63.6240],
        [165.5640, 105.9811],
        [139.1278,  84.7607],
        [121.6900,  78.4568],
        [134.3212,  85.3951],
        [101.7566,  63.6240],
        [130.2580,  81.5608],
        [130.4454,  80.3995],
        [125.3262,  80.9267],
        [112.4880,  69.1424]], grad_fn=<AddmmBackward0>)

In [119]:
targets

tensor([[120.,  75.],
        [140.,  80.],
        [110.,  65.],
        [130.,  85.],
        [115.,  70.],
        [125.,  95.],
        [145., 100.],
        [135.,  90.],
        [120.,  85.],
        [110.,  65.],
        [125.,  70.],
        [115.,  60.],
        [130.,  75.],
        [140.,  85.],
        [150., 100.]])